# Imputing Data on 2014

In [1]:
import pandas as pd
import numpy as np


#%matplotlib notebook to make interactive within same window as JN
#%matplotlib
#%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

#SciKit Learn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

from sklearn import preprocessing
from sklearn.decomposition import PCA

#Plotly
import plotly.express as px
import plotly.graph_objects as go

import seaborn as sns
import scipy as sci
import statsmodels as stats

## Data

In [11]:
# Data

WorldData = pd.read_csv("WorldMarketData.csv", na_values = '..')
df_original = pd.DataFrame(WorldData)
pd.options.display.float_format = '{:,.2f}'.format
df_original.columns = ['country', 'country_code', 'series', 'series_code', '1994', '1995', '1996', '1997', '1998', 
             '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', 
             '2012', '2013', '2014', '2015', '2016', '2017', '2018']

df = df_original.drop(columns=['country_code', 'series_code'])
df = df.drop(df.index[3800:])
df1 = pd.melt(df,
                       ["country", 'series'],
                       var_name="year",
                       value_name="value")
#df1 = df1.sort_values(by=["year"], ascending = True)
df1 = df1.set_index('country').drop(index = ['World', 'Low income', 'Middle income', 'High income'])
df1 = df1.reset_index()
df1.head()

,country,series,year,value
0,United States,Agricultural raw materials exports (% of merch...,1994,3.49
1,United States,Agricultural raw materials imports (% of merch...,1994,2.04
2,United States,Average time to clear exports through customs ...,1994,nan
3,United States,Bribery incidence (% of firms experiencing at ...,1994,nan
4,United States,Changes in inventories (current US$),1994,"63,785,000,000.00"


In [13]:
df_stack = df1.set_index(['series', 'country'])
df_stack_all = df_stack

dfsa = df_stack_all.reset_index()
dfsa = dfsa.pivot_table(index = ['country', 'year'], columns = 'series', values='value')
dfsa = dfsa.reset_index()
dfsa = dfsa.dropna(thresh=len(dfsa) - 1000, axis=1)

dfsa14 = dfsa[dfsa['year'] == '2014']
dfsa14 = dfsa14.drop(columns = ['year'])
dfsa14.set_index('country', inplace = True)

dfsa14.head()

series,Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),Changes in inventories (current US$),Commercial service exports (current US$),Commercial service imports (current US$),"Computer, communications and other services (% of commercial service exports)","Computer, communications and other services (% of commercial service imports)",Cost of business start-up procedures (% of GNI per capita),"Cost to export, border compliance (US$)","Cost to export, documentary compliance (US$)",...,"Tariff rate, applied, simple mean, all products (%)","Tariff rate, most favored nation, weighted mean, all products (%)",Technicians in R&D (per million people),Time required to build a warehouse (days),Time required to start a business (days),"Time to export, border compliance (hours)","Time to export, documentary compliance (hours)","Time to import, border compliance (hours)","Time to import, documentary compliance (hours)",Trade (% of GDP)
country,,,,,,,,,,,,,,,,,,,,,
Argentina,1.07,0.89,"6,752,296,832.26","13,189,837,804.76","17,628,881,981.13",43.11,37.89,17.70,150.00,60.00,...,12.58,11.50,318.81,342.50,24.50,21.00,30.00,60.00,336.00,28.41
Australia,2.58,0.68,"-2,035,251,996.70","55,610,991,575.62","66,749,264,533.55",22.60,28.32,0.70,749.00,264.00,...,2.84,2.69,nan,112.00,2.50,36.00,7.00,37.00,3.00,42.51
Bahrain,0.03,1.04,"414,893,617.02","8,571,276,595.74","6,764,414,893.62",25.26,15.41,0.80,47.00,100.00,...,3.75,3.99,17.21,174.00,9.30,79.00,24.00,54.00,84.00,175.56
Belgium,1.39,1.27,"1,714,139,993.32","123,026,103,755.97","117,578,947,480.15",59.37,52.46,5.20,0.00,0.00,...,2.58,2.81,nan,212.00,4.00,0.00,1.00,0.00,1.00,164.70
Brazil,4.01,0.99,"16,587,335,316.62","39,046,701,888.60","85,915,546,867.55",62.78,50.47,5.50,862.00,226.40,...,13.74,9.92,nan,434.00,83.60,61.00,30.00,51.10,146.10,24.69


In [5]:
DF = dfsa14.copy()
fill_NaN = SimpleImputer(missing_values=np.nan, strategy='median')
imputed_DF = pd.DataFrame(fill_NaN.fit_transform(DF))
imputed_DF.columns = DF.columns
imputed_DF.index = DF.index
#Imputes the mean for the series at 2014, not for the country over the years...
#Because this is just for 2014, we will press on with these values, even though they are wildly innacurate.

In [15]:
DF.isnull().sum().sum() 

196

In [16]:
#All NaN values have been imputed
imputed_DF.isnull().sum().sum() 

0